In [215]:
import pandas as pd

def load_data(url):
    # read csv from github url
    # return pandas dataframe
    df = pd.read_csv(url)
    return df

# load inspections dataset
los_angeles_inspections_url = 'https://raw.githubusercontent.com/prathikr/CS_499_Final_Project/master/LOS_ANGELES_COUNTY_RESTAURANT_AND_MARKET_INSPECTIONS.csv'

df = load_data(los_angeles_inspections_url)
df.head()
print(df['State'])

,ACTIVITY DATE,OWNER ID,OWNER NAME,FACILITY ID,FACILITY NAME,RECORD ID,PROGRAM NAME,PROGRAM STATUS,PROGRAM ELEMENT (PE),PE DESCRIPTION,...,FACILITY CITY,FACILITY STATE,FACILITY ZIP,SERVICE CODE,SERVICE DESCRIPTION,SCORE,GRADE,SERIAL NUMBER,EMPLOYEE ID,Location
0,04/12/2017,OW0003351,AUDREY KIM BRIDAL COLLECTION,FA0010275,BOBA YA,PR0029878,BOBA YA,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,...,LA MIRADA,CA,90638,1,ROUTINE INSPECTION,95,A,DAH2Z9080,EE0000744,NaN
1,04/12/2017,OW0181132,THAI CITY DUARTE INC.,FA0174471,THAI CITY RESTAURANT,PR0167479,THAI CITY RESTAURANT,ACTIVE,1635,RESTAURANT (31-60) SEATS HIGH RISK,...,DUARTE,CA,91010,1,ROUTINE INSPECTION,90,A,DACH7Y18F,EE0000120,NaN
2,04/13/2017,OW0125502,SWEE KOK CHER,FA0160544,SUSHI FIRE,PR0148634,SUSHI FIRE,ACTIVE,1635,RESTAURANT (31-60) SEATS HIGH RISK,...,DUARTE,CA,91010,1,ROUTINE INSPECTION,90,A,DAEOLNF0Z,EE0000120,NaN
3,04/13/2017,OW0010326,DOS MONTANAS LLC,FA0012454,TIPSY WINGS,PR0018288,TIPSY WINGS,INACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,EL MONTE,CA,91732,1,ROUTINE INSPECTION,95,A,DABTUPVHJ,EE0000188,POINT (-118.020082 34.07441)
4,04/19/2017,OW0107342,MARIELENA RODRIGUEZ,FA0144532,ARI'S SHOP,PR0130618,ARI'S SHOP,ACTIVE,1610,"FOOD MKT RETAIL (1-1,999 SF) LOW RISK",...,LOS ANGELES,CA,90012,1,ROUTINE INSPECTION,100,A,DAL4G6OJH,EE0001058,NaN


In [216]:
print(df.columns)

Index(['ACTIVITY DATE', 'OWNER ID', 'OWNER NAME', 'FACILITY ID',
       'FACILITY NAME', 'RECORD ID', 'PROGRAM NAME', 'PROGRAM STATUS',
       'PROGRAM ELEMENT (PE)', 'PE DESCRIPTION', 'FACILITY ADDRESS',
       'FACILITY CITY', 'FACILITY STATE', 'FACILITY ZIP', 'SERVICE CODE',
       'SERVICE DESCRIPTION', 'SCORE', 'GRADE', 'SERIAL NUMBER', 'EMPLOYEE ID',
       'Location'],
      dtype='object')


In [217]:
# convert activity_date string to datetime object
df['ACTIVITY DATE'] = pd.to_datetime(df['ACTIVITY DATE'], format="%m/%d/%Y")
df['ACTIVITY DATE'].head()

0   2017-04-12
1   2017-04-12
2   2017-04-13
3   2017-04-13
4   2017-04-19
Name: ACTIVITY DATE, dtype: datetime64[ns]

In [218]:
# format all zipcodes into 5-digit numbers
df['FACILITY ZIP'] = df['FACILITY ZIP'].str[:5]
df['FACILITY ZIP'].head()

0    90638
1    91010
2    91010
3    91732
4    90012
Name: FACILITY ZIP, dtype: object

In [219]:
print("og df:", df.shape)
df.drop(columns=['PE DESCRIPTION'], inplace=True)
print("new df:", df.shape)

og df: (202339, 21)
new df: (202339, 20)


In [220]:
with open('ca_california_zip_codes.geojson', 'r') as jsonFile:
    data = json.load(jsonFile)
tmp = data

In [221]:
geozips = []
for i in range(len(tmp['features'])):
    if tmp['features'][i]['properties']['ZCTA5CE10'] in list(df['FACILITY ZIP'].unique()):
        geozips.append(tmp['features'][i])

In [222]:
new_json = dict.fromkeys(['type', 'features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

In [223]:
open("updated-file.json", "w").write(
    json.dumps(new_json, sort_keys=True, indent=4, separators=(',',': '))
)

24982514

In [224]:
def count_distribution(df, location, subgroup):
    group_counts = pd.DataFrame(df.groupby([location, subgroup]).size().unstack(1))
    group_counts.reset_index(inplace=True)
    return group_counts

In [225]:
def subgroup_distribution(df, location, subgroup):
    group = df.groupby([location, subgroup]).size()
    group_pcts = group.groupby(level=0).apply(lambda a: 100 * x/float(x.sum()))
    group_pcts = pd.DataFrame(group_pcts.unstack(1))
    group_pcts.reset_index(inplace=True)
    return group_pcts

In [226]:
import folium

def create_map(table, zips, mapped_feature, add_text=''):
    la_geo = r'updated-file.json'
    m = folium.Map(location=[34.0522, -118.2437], zoom_start=11)
    m.choropleth(
        geo_data = la_geo,
        fill_opacity = 0.7,
        line_opacity = 0.2,
        data = table,
        key_on = 'feature.properties.ZCTA5CE10',
        columns = [zips, mapped_feature],
        fill_color = 'RdYlGn',
        legend_name = (' ').join(mapped_feature.split('_')).title() + ' ' + add_text + ' Across LA'
    )
    folium.LayerControl().add_to(m)
    m.save(outfile=mapped_feature + "_map.html")

In [227]:
create_map(df, 'FACILITY ZIP', 'SCORE')